<a href="https://colab.research.google.com/github/MittalNeha/Extensive_Vision_AI6/blob/main/week4/EVA6_Session_4_bhaskar_squeeze_n_expand2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 12, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(12, 24, 3, padding=1)    #28 - 28 - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                 #28 - 14 - 10x10
        self.conv3 = nn.Conv2d(24, 48, 3, padding=1)   #14 - 14 - 12x12
        self.conv4 = nn.Conv2d(48, 96, 3, padding=1)  #14 - 14 - 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                 #14 -  7 - 28x28
        self.conv5 = nn.Conv2d(96, 192, 3)             #7  -  5 - 30x30
        self.conv6 = nn.Conv2d(192, 384, 3)            #5 -   3 - 32x32
        self.conv7 = nn.Conv2d(384, 10, 3)             #3 -   1 - 34x34
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             120
            Conv2d-2           [-1, 24, 28, 28]           2,616
         MaxPool2d-3           [-1, 24, 14, 14]               0
            Conv2d-4           [-1, 48, 14, 14]          10,416
            Conv2d-5           [-1, 96, 14, 14]          41,568
         MaxPool2d-6             [-1, 96, 7, 7]               0
            Conv2d-7            [-1, 192, 5, 5]         166,080
            Conv2d-8            [-1, 384, 3, 3]         663,936
            Conv2d-9             [-1, 10, 1, 1]          34,570
Total params: 919,306
Trainable params: 919,306
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.57
Params size (MB): 3.51
Estimated Total Size (MB): 4.08
-------------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))          #In future check why this normalization works 
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)   #send data to device/GPU
        optimizer.zero_grad()                               #reset to zero gradient
        output = model(data)                                #apply model
        loss = F.nll_loss(output, target)                   #get loss
        loss.backward()                                     #propagate loss backward
        optimizer.step()                                    
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),   #adjusted the decimal place to print finer accuracy
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0777, Accuracy: 9739/10000 (97.39%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0438, Accuracy: 9860/10000 (98.60%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0299, Accuracy: 9907/10000 (99.07%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0312, Accuracy: 9915/10000 (99.15%)



loss=0.017463156953454018 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.62it/s]



Test set: Average loss: 0.0335, Accuracy: 9905/10000 (99.05%)



In [ ]:
class Net2(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net2, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)    #28 - 28 - 5x5
        self.conv2_1 = nn.Conv2d(64, 32, 1)    #28 - 28 - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                 #28 - 14 - 10x10
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)   #14 - 14 - 12x12
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)  #14 - 14 - 14x14
        self.conv4_1 = nn.Conv2d(128, 32, 1)
        self.pool2 = nn.MaxPool2d(2, 2)                 #14 -  7 - 28x28
        self.conv5 = nn.Conv2d(32, 64, 3)             #7  -  5 - 30x30
        self.conv6 = nn.Conv2d(64, 128, 3)            #5 -   3 - 32x32
        self.conv7 = nn.Conv2d(128, 10, 3)             #3 -   1 - 34x34
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x)))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [ ]:
model = Net2().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
            Conv2d-3           [-1, 32, 28, 28]           2,080
         MaxPool2d-4           [-1, 32, 14, 14]               0
            Conv2d-5           [-1, 64, 14, 14]          18,496
            Conv2d-6          [-1, 128, 14, 14]          73,856
            Conv2d-7           [-1, 32, 14, 14]           4,128
         MaxPool2d-8             [-1, 32, 7, 7]               0
            Conv2d-9             [-1, 64, 5, 5]          18,496
           Conv2d-10            [-1, 128, 3, 3]          73,856
           Conv2d-11             [-1, 10, 1, 1]          11,530
Total params: 221,258
Trainable params: 221,258
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1321, Accuracy: 9580/10000 (95.80%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0647, Accuracy: 9794/10000 (97.94%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0425, Accuracy: 9877/10000 (98.77%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0425, Accuracy: 9872/10000 (98.72%)



loss=0.001070643775165081 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 38.15it/s]



Test set: Average loss: 0.0370, Accuracy: 9889/10000 (98.89%)



In [ ]:
class Net3(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net3, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 24, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(24, 48, 3)               #28 - 26 - 5x5
        self.conv2_1 = nn.Conv2d(48, 64, 3)             #26 - 24 - 7x7
        self.conv2_2 = nn.Conv2d(64, 24, 1)             #24 - 24 - 7x7
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 14x14
        self.conv3 = nn.Conv2d(24, 48, 3, padding=1)   #12 - 12 - 12x12
        self.conv4 = nn.Conv2d(48, 64, 3)              #12 - 10 - 14x14
        self.conv4_1 = nn.Conv2d(64, 24, 1)           #10 - 10 - 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                 #10 -  5 - 28x28
        self.conv5 = nn.Conv2d(24, 48, 3)             #5  -  3 - 30x30
        self.conv6 = nn.Conv2d(48, 10, 3)            #3 -   1 - 32x32
        #self.conv7 = nn.Conv2d(128, 10, 3)             #3 -   1 - 34x34
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2_2(F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x)))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = (self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [ ]:
model = Net3().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 28, 28]             240
            Conv2d-2           [-1, 48, 26, 26]          10,416
            Conv2d-3           [-1, 64, 24, 24]          27,712
            Conv2d-4           [-1, 24, 24, 24]           1,560
         MaxPool2d-5           [-1, 24, 12, 12]               0
            Conv2d-6           [-1, 48, 12, 12]          10,416
            Conv2d-7           [-1, 64, 10, 10]          27,712
            Conv2d-8           [-1, 24, 10, 10]           1,560
         MaxPool2d-9             [-1, 24, 5, 5]               0
           Conv2d-10             [-1, 48, 3, 3]          10,416
           Conv2d-11             [-1, 10, 1, 1]           4,330
Total params: 94,362
Trainable params: 94,362
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1397, Accuracy: 9575/10000 (95.75%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0683, Accuracy: 9776/10000 (97.76%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0389, Accuracy: 9882/10000 (98.82%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0383, Accuracy: 9886/10000 (98.86%)



loss=0.0018614205764606595 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 37.63it/s]



Test set: Average loss: 0.0382, Accuracy: 9893/10000 (98.93%)



In [ ]:
class Net4(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net4, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 24, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(24, 36, 3)               #28 - 26 - 5x5
        self.conv2_1 = nn.Conv2d(36, 48, 3)             #26 - 24 - 7x7
        self.conv2_2 = nn.Conv2d(48, 24, 1)             #24 - 24 - 7x7
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 14x14
        self.conv3 = nn.Conv2d(24, 36, 3, padding=1)               #12 - 10 - 12x12
        self.conv4 = nn.Conv2d(36, 48, 3)              #10 - 8 - 14x14
        self.conv4_1 = nn.Conv2d(48, 24, 1)           #8 - 8 - 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(24, 48, 3)             #4  -  2 - 30x30
        self.conv6 = nn.Conv2d(48, 10, 3)            #3 -   1 - 32x32
        #self.conv7 = nn.Conv2d(128, 10, 3)             #3 -   1 - 34x34
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2_2(F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x)))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = (self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [ ]:
model = Net4().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 28, 28]             240
            Conv2d-2           [-1, 36, 26, 26]           7,812
            Conv2d-3           [-1, 48, 24, 24]          15,600
            Conv2d-4           [-1, 24, 24, 24]           1,176
         MaxPool2d-5           [-1, 24, 12, 12]               0
            Conv2d-6           [-1, 36, 12, 12]           7,812
            Conv2d-7           [-1, 48, 10, 10]          15,600
            Conv2d-8           [-1, 24, 10, 10]           1,176
         MaxPool2d-9             [-1, 24, 5, 5]               0
           Conv2d-10             [-1, 48, 3, 3]          10,416
           Conv2d-11             [-1, 10, 1, 1]           4,330
Total params: 64,162
Trainable params: 64,162
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.3086, Accuracy: 1028/10000 (10.28%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1736, Accuracy: 9459/10000 (94.59%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0388, Accuracy: 9878/10000 (98.78%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0388, Accuracy: 9871/10000 (98.71%)



loss=0.007338423747569323 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.20it/s]



Test set: Average loss: 0.0326, Accuracy: 9893/10000 (98.93%)



In [ ]:
class Net5(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net5, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 24, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(24, 24, 3)               #28 - 26 - 5x5
        self.conv2_1 = nn.Conv2d(24, 36, 3)             #26 - 24 - 7x7
        self.conv2_2 = nn.Conv2d(36, 24, 1)             #24 - 24 - 7x7
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 14x14
        self.conv3 = nn.Conv2d(24, 24, 3, padding=1)               #12 - 10 - 12x12
        self.conv4 = nn.Conv2d(24, 36, 3)              #10 - 8 - 14x14
        self.conv4_1 = nn.Conv2d(36, 24, 1)           #8 - 8 - 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(24, 36, 3)             #4  -  2 - 30x30
        self.conv6 = nn.Conv2d(36, 10, 3)            #3 -   1 - 32x32
        #self.conv7 = nn.Conv2d(128, 10, 3)             #3 -   1 - 34x34
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2_2(F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x)))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = (self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [ ]:
model = Net5().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 28, 28]             240
            Conv2d-2           [-1, 24, 26, 26]           5,208
            Conv2d-3           [-1, 36, 24, 24]           7,812
            Conv2d-4           [-1, 24, 24, 24]             888
         MaxPool2d-5           [-1, 24, 12, 12]               0
            Conv2d-6           [-1, 24, 12, 12]           5,208
            Conv2d-7           [-1, 36, 10, 10]           7,812
            Conv2d-8           [-1, 24, 10, 10]             888
         MaxPool2d-9             [-1, 24, 5, 5]               0
           Conv2d-10             [-1, 36, 3, 3]           7,812
           Conv2d-11             [-1, 10, 1, 1]           3,250
Total params: 39,118
Trainable params: 39,118
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.3097, Accuracy: 9039/10000 (90.39%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0787, Accuracy: 9750/10000 (97.50%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0347, Accuracy: 9884/10000 (98.84%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0347, Accuracy: 9893/10000 (98.93%)



loss=0.025540096685290337 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 36.92it/s]



Test set: Average loss: 0.0382, Accuracy: 9872/10000 (98.72%)



In [ ]:
class Net6(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net6, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 24, 3)                #28 - 26 - 3x3
        self.conv2 = nn.Conv2d(24, 36, 3)               #26 - 24 - 5x5
        self.conv2_1 = nn.Conv2d(36, 48, 3)             #24 - 22 - 7x7
        self.conv2_2 = nn.Conv2d(48, 24, 1)             #22 - 22 - 7x7
        self.pool1 = nn.MaxPool2d(2, 2)                 #22 - 11 - 14x14
        self.conv3 = nn.Conv2d(24, 36, 3)              #11 - 9 - 12x12
        self.conv4 = nn.Conv2d(36, 48, 3)              #9 - 7 - 14x14
        self.conv4_1 = nn.Conv2d(48, 24, 1)           #7 - 7 - 14x14
        #self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(24, 48, 3)             #7  -  5 - 30x30
        #self.conv6 = nn.Conv2d(36, 48, 3)            #5 -   3 - 32x32
        #self.conv7 = nn.Conv2d(48, 10, 3)             #3 -   1 - 34x34
        self.gap = nn.AvgPool2d(kernel_size=5)
        self.fc1 = nn.Linear(48, 10)
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2_2(F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x)))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = (F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        x = F.relu(self.conv5(x))
        #print(x.shape)
        x = self.gap(x)
        #print(x.shape)
        x = x.view(-1, 1*1*48)
        x = self.fc1(x)
        #print(x.shape)
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [ ]:
model = Net6().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 26, 26]             240
            Conv2d-2           [-1, 36, 24, 24]           7,812
            Conv2d-3           [-1, 48, 22, 22]          15,600
            Conv2d-4           [-1, 24, 22, 22]           1,176
         MaxPool2d-5           [-1, 24, 11, 11]               0
            Conv2d-6             [-1, 36, 9, 9]           7,812
            Conv2d-7             [-1, 48, 7, 7]          15,600
            Conv2d-8             [-1, 24, 7, 7]           1,176
            Conv2d-9             [-1, 48, 5, 5]          10,416
        AvgPool2d-10             [-1, 48, 1, 1]               0
           Linear-11                   [-1, 10]             490
Total params: 60,322
Trainable params: 60,322
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)



  0%|          | 0/469 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb578bee710>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


loss=2.315734624862671 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=2.315734624862671 batch_id=0:   0%|          | 1/469 [00:00<01:09,  6.76it/s]

loss=2.30916738510


Test set: Average loss: 2.3010, Accuracy: 1135/10000 (11.35%)





loss=2.2977044582366943 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=2.3062148094177246 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]

loss=2.3062148094177246 batch_id=1:   0%|          | 2/469 [00:00<00:29, 15.67it/s]

loss=2.305025577545166 batch_id=2:   0%|          | 2/469 [00:00<00:29, 15.67it/s] 

loss=2.3058273792266846 batch_id=3:   0%|          | 2/469 [00:00<00:29, 15.67it/s]

loss=2.300696849822998 batch_id=4:   0%|          | 2/469 [00:00<00:29, 15.67it/s] 

loss=2.300696849822998 batch_id=4:   1%|          | 5/469 [00:00<00:25, 17.89it/s]

loss=2.3106892108917236 batch_id=5:   1%|          | 5/469 [00:00<00:25, 17.89it/s]

loss=2.3018436431884766 batch_id=6:   1%|          | 5/469 [00:00<00:25, 17.89it/s]

loss=2.309453248977661 batch_id=7:   1%|          | 5/469 [00:00<00:25, 17.89it/s] 

loss=2.305785655975342 batch_id=8:   1%|          | 5/469 [00:00<00:25, 17.89it/s]

loss=2.305785655975342 batch_id=8:   2%|▏         | 9/469 [00:00<00:22, 20.83it/s


Test set: Average loss: 2.2996, Accuracy: 1135/10000 (11.35%)





loss=2.2977755069732666 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=2.2977755069732666 batch_id=0:   0%|          | 1/469 [00:00<00:53,  8.82it/s]

loss=2.2970752716064453 batch_id=1:   0%|          | 1/469 [00:00<00:53,  8.82it/s]

loss=2.295999526977539 batch_id=2:   0%|          | 1/469 [00:00<00:53,  8.82it/s] 

loss=2.2969141006469727 batch_id=3:   0%|          | 1/469 [00:00<00:53,  8.82it/s]

loss=2.2969141006469727 batch_id=3:   1%|          | 4/469 [00:00<00:42, 11.01it/s]

loss=2.2956385612487793 batch_id=4:   1%|          | 4/469 [00:00<00:42, 11.01it/s]

loss=2.307393789291382 batch_id=5:   1%|          | 4/469 [00:00<00:42, 11.01it/s] 

loss=2.2998898029327393 batch_id=6:   1%|          | 4/469 [00:00<00:42, 11.01it/s]

loss=2.298041582107544 batch_id=7:   1%|          | 4/469 [00:00<00:42, 11.01it/s] 

loss=2.298041582107544 batch_id=7:   2%|▏         | 8/469 [00:00<00:34, 13.52it/s]

loss=2.3036296367645264 batch_id=8:   2%|▏         | 8/469 [00:00<00:34,

...


Test set: Average loss: 0.0649, Accuracy: 9788/10000 (97.88%)





loss=0.04251866042613983 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.04251866042613983 batch_id=0:   0%|          | 1/469 [00:00<00:58,  8.01it/s]

loss=0.043933793902397156 batch_id=1:   0%|          | 1/469 [00:00<00:58,  8.01it/s]

loss=0.061657171696424484 batch_id=2:   0%|          | 1/469 [00:00<00:58,  8.01it/s]

loss=0.026236040517687798 batch_id=3:   0%|          | 1/469 [00:00<00:58,  8.01it/s]

loss=0.026236040517687798 batch_id=3:   1%|          | 4/469 [00:00<00:46, 10.09it/s]

loss=0.1091524064540863 batch_id=4:   1%|          | 4/469 [00:00<00:46, 10.09it/s]  

loss=0.07938013970851898 batch_id=5:   1%|          | 4/469 [00:00<00:46, 10.09it/s]

loss=0.07938013970851898 batch_id=5:   1%|▏         | 6/469 [00:00<00:39, 11.70it/s]

loss=0.05139714479446411 batch_id=6:   1%|▏         | 6/469 [00:00<00:39, 11.70it/s]

loss=0.045988183468580246 batch_id=7:   1%|▏         | 6/469 [00:00<00:39, 11.70it/s]

loss=0.045988183468580246 batch_id=7:   2%|▏         |


Test set: Average loss: 0.0546, Accuracy: 9814/10000 (98.14%)





loss=0.12346450984477997 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.12346450984477997 batch_id=0:   0%|          | 1/469 [00:00<01:02,  7.48it/s]

loss=0.12585075199604034 batch_id=1:   0%|          | 1/469 [00:00<01:02,  7.48it/s]

loss=0.022221099585294724 batch_id=2:   0%|          | 1/469 [00:00<01:02,  7.48it/s]

loss=0.10731948167085648 batch_id=3:   0%|          | 1/469 [00:00<01:02,  7.48it/s] 

loss=0.10731948167085648 batch_id=3:   1%|          | 4/469 [00:00<00:49,  9.39it/s]

loss=0.04451065883040428 batch_id=4:   1%|          | 4/469 [00:00<00:49,  9.39it/s]

loss=0.01444123312830925 batch_id=5:   1%|          | 4/469 [00:00<00:49,  9.39it/s]

loss=0.05617310106754303 batch_id=6:   1%|          | 4/469 [00:00<00:49,  9.39it/s]

loss=0.05617310106754303 batch_id=6:   1%|▏         | 7/469 [00:00<00:39, 11.68it/s]

loss=0.1361106038093567 batch_id=7:   1%|▏         | 7/469 [00:00<00:39, 11.68it/s] 

loss=0.02464432828128338 batch_id=8:   1%|▏         | 7/46


Test set: Average loss: 0.0465, Accuracy: 9844/10000 (98.44%)



In [8]:
class Net7(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net7, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 16, 3)                #28 - 26 - 3x3
        self.conv2 = nn.Conv2d(16, 32, 3)               #26 - 24 - 5x5
        self.conv2_1 = nn.Conv2d(32, 16, 1)             #24 - 24 - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 10x10
        self.conv3 = nn.Conv2d(16, 16, 3)              #12 - 10 - 12x12
        self.conv4 = nn.Conv2d(16, 32, 3)              #10 - 8 - 14x14
        self.conv4_1 = nn.Conv2d(32, 16, 1)           #8 - 8 - 14x14
        #self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(16, 16, 3)             #8  -  6 - 16x16
        #self.conv6 = nn.Conv2d(36, 48, 3)            #5 -   3 - 32x32
        #self.conv7 = nn.Conv2d(48, 10, 3)             #3 -   1 - 34x34
        self.gap = nn.AvgPool2d(kernel_size=6)
        self.fc1 = nn.Linear(16, 10)
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1((F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = (F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        x = F.relu(self.conv5(x))
        #print(x.shape)
        x = self.gap(x)
        #print(x.shape)
        x = x.view(-1, 1*1*16)
        x = self.fc1(x)
        #print(x.shape)
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [9]:
model = Net7().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
            Conv2d-2           [-1, 32, 24, 24]           4,640
            Conv2d-3           [-1, 16, 24, 24]             528
         MaxPool2d-4           [-1, 16, 12, 12]               0
            Conv2d-5           [-1, 16, 10, 10]           2,320
            Conv2d-6             [-1, 32, 8, 8]           4,640
            Conv2d-7             [-1, 16, 8, 8]             528
            Conv2d-8             [-1, 16, 6, 6]           2,320
         AvgPool2d-9             [-1, 16, 1, 1]               0
           Linear-10                   [-1, 10]             170
Total params: 15,306
Trainable params: 15,306
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.35
Params size (MB): 0.06
Estimated Tot

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.3010, Accuracy: 1135/10000 (11.35%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.2931, Accuracy: 2001/10000 (20.01%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0773, Accuracy: 9752/10000 (97.52%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0947, Accuracy: 9721/10000 (97.21%)



loss=0.09830998629331589 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 38.67it/s]



Test set: Average loss: 0.0574, Accuracy: 9824/10000 (98.24%)



In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

total_epochs = 5
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0571, Accuracy: 9833/10000 (98.33%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0529, Accuracy: 9831/10000 (98.31%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0461, Accuracy: 9842/10000 (98.42%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0470, Accuracy: 9855/10000 (98.55%)



loss=0.05478796735405922 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.92it/s]



Test set: Average loss: 0.0449, Accuracy: 9852/10000 (98.52%)



In [12]:
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.5)

total_epochs = 5
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0436, Accuracy: 9852/10000 (98.52%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0429, Accuracy: 9858/10000 (98.58%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0425, Accuracy: 9857/10000 (98.57%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0428, Accuracy: 9858/10000 (98.58%)



loss=0.03924887999892235 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.81it/s]



Test set: Average loss: 0.0425, Accuracy: 9860/10000 (98.60%)



In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.5)

total_epochs = 5
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0443, Accuracy: 9851/10000 (98.51%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0425, Accuracy: 9863/10000 (98.63%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0416, Accuracy: 9857/10000 (98.57%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0416, Accuracy: 9859/10000 (98.59%)



loss=0.008166328072547913 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 40.06it/s]



Test set: Average loss: 0.0430, Accuracy: 9867/10000 (98.67%)



In [14]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

total_epochs = 5
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0485, Accuracy: 9849/10000 (98.49%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0479, Accuracy: 9852/10000 (98.52%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0465, Accuracy: 9845/10000 (98.45%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0428, Accuracy: 9853/10000 (98.53%)



loss=0.0559074766933918 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 40.63it/s]



Test set: Average loss: 0.0414, Accuracy: 9868/10000 (98.68%)



In [21]:
class Net8(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net8, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 16, 3)                #28 - 26 - 3x3
        self.conv2 = nn.Conv2d(16, 32, 3)               #26 - 24 - 5x5
        self.conv2_1 = nn.Conv2d(32, 16, 1)             #24 - 24 - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 10x10
        self.conv3 = nn.Conv2d(16, 16, 3)              #12 - 10 - 12x12
        self.conv4 = nn.Conv2d(16, 32, 3)              #10 - 8 - 14x14
        self.conv4_1 = nn.Conv2d(32, 16, 1)           #8 - 8 - 14x14
        #self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(16, 32, 3)             #8  -  6 - 16x16
        #self.conv6 = nn.Conv2d(36, 48, 3)            #5 -   3 - 32x32
        #self.conv7 = nn.Conv2d(48, 10, 3)             #3 -   1 - 34x34
        self.gap = nn.AvgPool2d(kernel_size=6)
        self.fc1 = nn.Linear(32, 10)
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1((F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = (F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        x = F.relu(self.conv5(x))
        #print(x.shape)
        x = self.gap(x)
        #print(x.shape, x.shape[0])
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = self.fc1(x)
        #print(x.shape)
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [22]:
model = Net8().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
            Conv2d-2           [-1, 32, 24, 24]           4,640
            Conv2d-3           [-1, 16, 24, 24]             528
         MaxPool2d-4           [-1, 16, 12, 12]               0
            Conv2d-5           [-1, 16, 10, 10]           2,320
            Conv2d-6             [-1, 32, 8, 8]           4,640
            Conv2d-7             [-1, 16, 8, 8]             528
            Conv2d-8             [-1, 32, 6, 6]           4,640
         AvgPool2d-9             [-1, 32, 1, 1]               0
           Linear-10                   [-1, 10]             330
Total params: 17,786
Trainable params: 17,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.36
Params size (MB): 0.07
Estimated Tot

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [23]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.3011, Accuracy: 1135/10000 (11.35%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.3011, Accuracy: 1135/10000 (11.35%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0967, Accuracy: 9708/10000 (97.08%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0681, Accuracy: 9788/10000 (97.88%)



loss=0.09720372408628464 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 40.22it/s]



Test set: Average loss: 0.0651, Accuracy: 9782/10000 (97.82%)



In [24]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0474, Accuracy: 9841/10000 (98.41%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0499, Accuracy: 9844/10000 (98.44%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0379, Accuracy: 9879/10000 (98.79%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0384, Accuracy: 9885/10000 (98.85%)



loss=0.10723763704299927 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.26it/s]



Test set: Average loss: 0.0397, Accuracy: 9880/10000 (98.80%)



In [25]:
class Net9(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net9, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 16, 3)                #28 - 26 - 3x3
        self.conv2 = nn.Conv2d(16, 32, 3)               #26 - 24 - 5x5
        self.conv2_1 = nn.Conv2d(32, 16, 1)             #24 - 24 - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 10x10
        self.conv3 = nn.Conv2d(16, 32, 3)              #12 - 10 - 12x12
        self.conv4 = nn.Conv2d(32, 32, 3)              #10 - 8 - 14x14
        self.conv4_1 = nn.Conv2d(32, 16, 1)           #8 - 8 - 14x14
        #self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(16, 32, 3)             #8  -  6 - 16x16
        #self.conv6 = nn.Conv2d(36, 48, 3)            #5 -   3 - 32x32
        #self.conv7 = nn.Conv2d(48, 10, 3)             #3 -   1 - 34x34
        self.gap = nn.AvgPool2d(kernel_size=6)
        self.fc1 = nn.Linear(32, 10)
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1((F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = (F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        x = F.relu(self.conv5(x))
        #print(x.shape)
        x = self.gap(x)
        #print(x.shape, x.shape[0])
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = self.fc1(x)
        #print(x.shape)
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [26]:
model = Net9().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
            Conv2d-2           [-1, 32, 24, 24]           4,640
            Conv2d-3           [-1, 16, 24, 24]             528
         MaxPool2d-4           [-1, 16, 12, 12]               0
            Conv2d-5           [-1, 32, 10, 10]           4,640
            Conv2d-6             [-1, 32, 8, 8]           9,248
            Conv2d-7             [-1, 16, 8, 8]             528
            Conv2d-8             [-1, 32, 6, 6]           4,640
         AvgPool2d-9             [-1, 32, 1, 1]               0
           Linear-10                   [-1, 10]             330
Total params: 24,714
Trainable params: 24,714
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.37
Params size (MB): 0.09
Estimated Tot

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [27]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.3010, Accuracy: 1135/10000 (11.35%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.3003, Accuracy: 1135/10000 (11.35%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0613, Accuracy: 9820/10000 (98.20%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0576, Accuracy: 9835/10000 (98.35%)



loss=0.0882435217499733 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.45it/s]



Test set: Average loss: 0.0506, Accuracy: 9837/10000 (98.37%)



In [28]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0539, Accuracy: 9821/10000 (98.21%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0400, Accuracy: 9884/10000 (98.84%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0396, Accuracy: 9873/10000 (98.73%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0325, Accuracy: 9900/10000 (99.00%)



loss=0.008994449861347675 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 40.66it/s]



Test set: Average loss: 0.0319, Accuracy: 9908/10000 (99.08%)



In [29]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

total_epochs = 5
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0343, Accuracy: 9900/10000 (99.00%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0301, Accuracy: 9903/10000 (99.03%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0299, Accuracy: 9907/10000 (99.07%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0293, Accuracy: 9904/10000 (99.04%)



loss=0.0446196012198925 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.71it/s]



Test set: Average loss: 0.0277, Accuracy: 9913/10000 (99.13%)



In [33]:
class Net7_1(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net7_1, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 16, 3)                #28 - 26 - 3x3
        self.conv2 = nn.Conv2d(16, 32, 3)               #26 - 24 - 5x5
        self.conv2_1 = nn.Conv2d(32, 16, 1)             #24 - 24 - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 10x10
        self.conv3 = nn.Conv2d(16, 16, 3)              #12 - 10 - 12x12
        self.conv4 = nn.Conv2d(16, 32, 3)              #10 - 8 - 14x14
        self.conv4_1 = nn.Conv2d(32, 16, 1)           #8 - 8 - 14x14
        #self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(16, 16, 3)             #8  -  6 - 16x16
        self.conv6 = nn.Conv2d(16, 32, 2)            #6 -   5 - 32x32
        #self.conv7 = nn.Conv2d(48, 10, 3)             #3 -   1 - 34x34
        self.gap = nn.AvgPool2d(kernel_size=5)
        self.fc1 = nn.Linear(32, 10)
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1((F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = (F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv5(x))
        #print(x.shape)
        x = self.gap(x)
        #print(x.shape)
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = self.fc1(x)
        #print(x.shape)
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)  

In [34]:
model = Net7_1().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
            Conv2d-2           [-1, 32, 24, 24]           4,640
            Conv2d-3           [-1, 16, 24, 24]             528
         MaxPool2d-4           [-1, 16, 12, 12]               0
            Conv2d-5           [-1, 16, 10, 10]           2,320
            Conv2d-6             [-1, 32, 8, 8]           4,640
            Conv2d-7             [-1, 16, 8, 8]             528
            Conv2d-8             [-1, 16, 6, 6]           2,320
            Conv2d-9             [-1, 32, 5, 5]           2,080
        AvgPool2d-10             [-1, 32, 1, 1]               0
           Linear-11                   [-1, 10]             330
Total params: 17,546
Trainable params: 17,546
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [35]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.3008, Accuracy: 1135/10000 (11.35%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.1889, Accuracy: 1986/10000 (19.86%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0623, Accuracy: 9799/10000 (97.99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0646, Accuracy: 9799/10000 (97.99%)



loss=0.023945072665810585 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.30it/s]



Test set: Average loss: 0.0583, Accuracy: 9820/10000 (98.20%)



In [36]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0469, Accuracy: 9868/10000 (98.68%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0449, Accuracy: 9868/10000 (98.68%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0428, Accuracy: 9876/10000 (98.76%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0463, Accuracy: 9857/10000 (98.57%)



loss=0.023763610050082207 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.17it/s]



Test set: Average loss: 0.0415, Accuracy: 9871/10000 (98.71%)



In [37]:
class Net7_2(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net7_2, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 16, 3)                #28 - 26 - 3x3
        self.conv2 = nn.Conv2d(16, 32, 3)               #26 - 24 - 5x5
        self.conv2_1 = nn.Conv2d(32, 16, 1)             #24 - 24 - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                 #24 - 12 - 10x10
        self.conv3 = nn.Conv2d(16, 16, 3)              #12 - 10 - 12x12
        self.conv4 = nn.Conv2d(16, 32, 3)              #10 - 8 - 14x14
        self.conv4_1 = nn.Conv2d(32, 16, 1)           #8 - 8 - 14x14
        #self.pool2 = nn.MaxPool2d(2, 2)                 #8 -  4 - 28x28
        self.conv5 = nn.Conv2d(16, 16, 3)             #8  -  6 - 16x16
        self.conv6 = nn.Conv2d(16, 32, 3)            #6 -   4 - 32x32
        #self.conv7 = nn.Conv2d(48, 10, 3)             #3 -   1 - 34x34
        self.gap = nn.AvgPool2d(kernel_size=4)
        self.fc1 = nn.Linear(32, 10)
        

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1((F.relu(self.conv2_1(F.relu(self.conv2(F.relu(self.conv1(x))))))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = (F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv5(x))
        #print(x.shape)
        x = self.gap(x)
        #print(x.shape)
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = self.fc1(x)
        #print(x.shape)
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        #x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)  

In [38]:
model = Net7_2().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
            Conv2d-2           [-1, 32, 24, 24]           4,640
            Conv2d-3           [-1, 16, 24, 24]             528
         MaxPool2d-4           [-1, 16, 12, 12]               0
            Conv2d-5           [-1, 16, 10, 10]           2,320
            Conv2d-6             [-1, 32, 8, 8]           4,640
            Conv2d-7             [-1, 16, 8, 8]             528
            Conv2d-8             [-1, 16, 6, 6]           2,320
            Conv2d-9             [-1, 32, 4, 4]           4,640
        AvgPool2d-10             [-1, 32, 1, 1]               0
           Linear-11                   [-1, 10]             330
Total params: 20,106
Trainable params: 20,106
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [39]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.3012, Accuracy: 1135/10000 (11.35%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.3010, Accuracy: 1135/10000 (11.35%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0835, Accuracy: 9762/10000 (97.62%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0662, Accuracy: 9803/10000 (98.03%)



loss=0.05995149537920952 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 40.91it/s]



Test set: Average loss: 0.0636, Accuracy: 9816/10000 (98.16%)



In [40]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0499, Accuracy: 9841/10000 (98.41%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0468, Accuracy: 9844/10000 (98.44%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0443, Accuracy: 9857/10000 (98.57%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0473, Accuracy: 9844/10000 (98.44%)



loss=0.006905754562467337 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 42.09it/s]



Test set: Average loss: 0.0420, Accuracy: 9867/10000 (98.67%)

